In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [2]:
# 데이터 불러오기
train  = pd.read_csv('./data/train.csv')
test  = pd.read_csv('./data/test.csv')

pd.set_option('display.max_columns', None)

In [3]:
train.drop_duplicates(inplace=True, keep='last')

In [4]:
# 값이 너무 커서 log 처리
train['intro_log'] = np.log(train['introelapse'])
train['test_log'] = np.log(train['testelapse'])
train['survey_log'] = np.log(train['surveyelapse'])

test['intro_log'] = np.log(test['introelapse'])
test['test_log'] = np.log(test['testelapse'])
test['survey_log'] = np.log(test['surveyelapse'])

In [5]:
train.drop(['country', 'index'], axis=1, inplace=True)
test.drop(['country', 'index'], axis=1, inplace=True)

In [6]:
for i,j in train['age'].iteritems():
    if j>100:
        train.loc[i, 'age'] = 100
        
for i,j in test['age'].iteritems():
    if j>100:
        test.loc[i, 'age'] = 100
        
for i,j in train['familysize'].iteritems():
    if j>10:
        train.loc[i, 'familysize'] = 10
        
for i,j in test['familysize'].iteritems():
    if j>10:
        test.loc[i, 'familysize'] = 10

In [7]:
val_col = train.drop('nerdiness', axis=1).columns

In [8]:
from sklearn.preprocessing import MinMaxScaler

train.fillna(0, inplace=True)
test.fillna(0, inplace=True)

In [9]:
train.drop(['introelapse', 'testelapse', 'surveyelapse'], axis=1, inplace=True)
test.drop(['introelapse', 'testelapse', 'surveyelapse'], axis=1, inplace=True)

In [10]:
from sklearn.model_selection import train_test_split

scaler = MinMaxScaler()

y = train['nerdiness']
x_train = train.loc[:, 'Q1':'ASD']
y_train = train['nerdiness']

# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

In [11]:
from sklearn.ensemble import RandomForestClassifier
rf_model=RandomForestClassifier(min_samples_split=3,min_samples_leaf=1,max_depth=160,max_features='auto')

In [12]:
from sklearn.model_selection import KFold, GridSearchCV #하이퍼파라미터 튜닝을 위한 불러오기
from sklearn.metrics import accuracy_score

In [13]:
params = {'n_estimators':[100, 1000, 3000]}
model_gs = GridSearchCV(rf_model, param_grid=params, scoring='accuracy', n_jobs=-1, verbose=1)
model_gs.fit(x_train, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  1.2min finished


GridSearchCV(estimator=RandomForestClassifier(max_depth=160,
                                              min_samples_split=3),
             n_jobs=-1, param_grid={'n_estimators': [100, 1000, 3000]},
             scoring='accuracy', verbose=1)

In [16]:
cv = KFold(n_splits=5, random_state=2022)

In [17]:
gcv = GridSearchCV(rf_model, param_grid=params, cv=cv, scoring='f1', n_jobs=-1)

In [18]:
gcv.fit(x_train.values, y_train.values)
print('best params', gcv.best_params_)

best params {'n_estimators': 3000}


### Bagging

In [19]:
rf_model = RandomForestClassifier(n_estimators=3000, random_state=2022, n_jobs=-1)

In [21]:
rf_model.fit(x_train, y_train)

RandomForestClassifier(n_estimators=3000, n_jobs=-1, random_state=2022)

In [30]:
x_test = test.loc[:, 'Q1':'ASD'] 
pred = rf_model.predict(x_test)

In [31]:
submission = pd.read_csv('./data/sample_submission.csv')

In [32]:
submission['nerdiness'] = pred

In [33]:
submission

,index,nerdiness
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1
...,...,...
35447,35447,1
35448,35448,1
35449,35449,1
35450,35450,0


In [34]:
submission.to_csv('랜포만함.csv', index=False)